In [1]:
# Improved cleaning of corrupted images from dataset
from PIL import Image, UnidentifiedImageError
import os

def clean_folder(folder):
    for root, dirs, files in os.walk(folder):
        for fname in files:
            if fname.lower().endswith('.jpg'):
                fpath = os.path.join(root, fname)
                try:
                    # Remove zero-byte files
                    if os.path.getsize(fpath) == 0:
                        print(f"Removing zero-byte image: {fpath}")
                        os.remove(fpath)
                        continue
                    # Try to fully open and load the image
                    with Image.open(fpath) as img:
                        img.load()
                except (OSError, UnidentifiedImageError, Exception):
                    print(f"Removing corrupted image: {fpath}")
                    os.remove(fpath)

clean_folder('/Users/rohanrao/Desktop/hair4face/womens faceshape/training_set')
clean_folder('/Users/rohanrao/Desktop/hair4face/womens faceshape/testing_set')

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Paths to training and testing sets
train_dir = '/Users/rohanrao/Desktop/hair4face/womens faceshape/training_set'
test_dir = '/Users/rohanrao/Desktop/hair4face/womens faceshape/testing_set'

# Image parameters
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Data generators (folders: Heart, Oblong, Oval, Round, Square)
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Model definition
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=IMG_SIZE + (3,)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(5, activation='softmax')  # 5 classes: Heart, Oblong, Oval, Round, Square
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data, epochs=10, validation_data=test_data)

# Evaluate
loss, acc = model.evaluate(test_data)
print(f'Test accuracy: {acc:.2f}')

Found 3998 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


/Users/rohanrao/Desktop/hair4face/.venv312/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/rohanrao/Desktop/hair4face/.venv312/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 26s 199ms/step - accuracy: 0.2364 - loss: 1.7422 - val_accuracy: 0.2910 - val_loss: 1.5676
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 24s 192ms/step - accuracy: 0.3672 - loss: 1.4885 - val_accuracy: 0.3350 - val_loss: 1.5192
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 28s 227ms/step - accuracy: 0.4632 - loss: 1.3110 - val_accuracy: 0.3630 - val_loss: 1.5492
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 27s 215ms/step - accuracy: 0.6126 - loss: 1.0065 - val_accuracy: 0.3550 - val_loss: 1.7766
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 30s 242ms/step - accuracy: 0.7821 - loss: 0.6150 - val_accuracy: 0.3490 - val_loss: 2.1544
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 227ms/step - accuracy: 0.9090 - loss: 0.2767 - val_accuracy: 0.3640 - val_loss: 2.6445
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 228ms/step - accuracy: 0.9697 - loss: 0.1126 - val_accuracy: 0.3570 - val_loss: 3.2455
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 230ms/step - accuracy: 0.9920 - loss: 0

In [3]:
# Save the trained model
model.save('codebase/women_faceshape_model.h5')

In [10]:
import random
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model = load_model('codebase/women_faceshape_model.h5')

# Get class labels from the training generator
class_labels = list(train_data.class_indices.keys())

# Pick a random image from the test set
test_dirs = [
    '/Users/rohanrao/Desktop/hair4face/womens faceshape/testing_set/Heart',
    '/Users/rohanrao/Desktop/hair4face/womens faceshape/testing_set/Oblong',
    '/Users/rohanrao/Desktop/hair4face/womens faceshape/testing_set/Oval',
    '/Users/rohanrao/Desktop/hair4face/womens faceshape/testing_set/Round',
    '/Users/rohanrao/Desktop/hair4face/womens faceshape/testing_set/Square'
]
test_dir = random.choice(test_dirs)
img_name = random.choice([f for f in os.listdir(test_dir) if f.lower().endswith('.jpg')])
img_path = os.path.join(test_dir, img_name)

# Load and preprocess the image
img = image.load_img(img_path, target_size=IMG_SIZE)
x = image.img_to_array(img)
x = x / 255.0
x = np.expand_dims(x, axis=0)

# Predict
pred = model.predict(x)
pred_class = class_labels[np.argmax(pred)]
print(f"Test image: {img_path}")
print(f"Predicted face shape: {pred_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Test image: /Users/rohanrao/Desktop/hair4face/womens faceshape/testing_set/Oval/oval (293).jpg
Predicted face shape: Oval
